In [ ]:
#
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# 1. Load data

The trainig dataset is too big and cannot be load into memory with default parameters of read_csv(...).

We can avoid running out of memory by using smaller datatypes for numbers. For example: instead of float64 (the default datatype for read_csv) we use float16.


https://www.kaggle.com/ilialar/ubiquant-eda-and-baseline

In [ ]:
# define wanted data types
data_types_dict = {
    'time_id': 'int16',
    'investment_id': 'int16',
    "target": 'float16',
}

## we don't do any processing of features -> no need to load them 
# features = [f'f_{i}' for i in range(300)]
# for f in features:
#     data_types_dict[f] = 'float16'

In [ ]:
%%time
train = pd.read_csv('/kaggle/input/ubiquant-market-prediction/train.csv',
                    usecols = data_types_dict.keys(),
                    dtype = data_types_dict)
display(train.info(),train.head())

In [ ]:
test = pd.read_csv('/kaggle/input/ubiquant-market-prediction/example_test.csv')
display(test.info(), test.head())

sample = pd.read_csv('/kaggle/input/ubiquant-market-prediction/example_sample_submission.csv')
display(sample.info(), sample.head())

# 2. Create Model

As the ambition of this notebook is just to make any prediction which is better than -1, we will use just simple averaging over investment ids.

In [ ]:
# average over investment_id
investment_means = train.groupby('investment_id')['target'].mean().to_dict()

# average over whole training dataset
train_mean = train['target'].mean()

Now we have a simple dictionary which links investment id to mean value of its target. We can use this dictionary to make the submission.

In [ ]:
def predict(prediction_df):
    # add unknown investment ids into dictionary
    for inv_id in prediction_df['investment_id']:
        investment_means.setdefault(inv_id,train_mean) # if not exist, add it with mean as value

    prediction = prediction_df.set_index('investment_id').index.map(investment_means.get)
    return prediction

Lets try if we can produce predictions for the testing dataframe.

In [ ]:
pred = predict(test)

pred

It works as expected - we have a vector of nine values (one per line of testing set) which represents our prediction of the target.

# 3. Submission

Let's use described algorithm to generate submission.

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df['target'] = predict(test_df)  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions

**Note:** For some reason the submission is not scored on leaderboard. I receive only "Submission Scoring Error" 😟

If you want to see very simple submission which receive valid score on leaderboard, check this one: https://www.kaggle.com/jiprud/simple-randomforest